In [67]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [68]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6705, 7)

In [69]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6692, 5)

In [70]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6711, 9)

fin empresas y comienzo de rescate de pacientes


In [71]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [ ]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1096578, 4)

In [ ]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1096578, 12)

Carga de Archivo carga

In [54]:
nuevos = ("F:/Empresas SF/reportes/carga.xlsx")
campaña ='Socofar'
Pais = 'Chile'




Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")

Pacientes_Archivo.head()                                       

,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,1° Apellido Asegurado,2° Apellido Asegurado,apellidos,nacimiento,Edad,email,direccion,comuna,celular,rut
0,Comercial Maicao SPA,6985407.0,9,NaN,NaN,6985407-9,Mujer,Titular,NaN,RUTH,FLORES,FUENZALIDA,FLORES FUENZALIDA,1957-01-13,66.0,6985407@casilladigital.cl,STUGUTTGARD 2112 Chile Decimo Quinta Region A...,ARICA,993894545,69854079
1,Comercial Maicao SPA,7141749.0,2,NaN,NaN,7141749-2,Mujer,Titular,NaN,INELIA,RODRIGUEZ,LEDESMA,RODRIGUEZ LEDESMA,1959-07-23,63.0,rodriguezinelia@gmail.com,AVDA. VALPARAISO CASA 1170 3 CHILE Región de ...,QUILLOTA,95966963,71417492
2,Comercial Maicao SPA,7196473.0,6,NaN,NaN,7196473-6,Mujer,Titular,NaN,JULIA,BURGOS,BURGOS,BURGOS BURGOS,1956-05-20,66.0,7196473@casilladigital.cl,GRAJALES 2461 30 CHILE Region Metropolitana d...,SANTIAGO,NaN,71964736
3,Comercial Maicao SPA,7476959.0,4,NaN,NaN,7476959-4,Mujer,Titular,NaN,MARISOL,BRAVO,LUARTE,BRAVO LUARTE,1955-12-02,67.0,7476959@casilladigital.cl,ECUADOR 1560 Chile Cuarta Region ELQUI LA SERENA,LA SERENA,998378146,74769594
4,Comercial Maicao SPA,7930009.0,8,NaN,NaN,7930009-8,Mujer,Titular,NaN,FRESIA EUGENIA,TAHORMINA,OLIVER,TAHORMINA OLIVER,1958-12-19,64.0,7930009@CASILLADIGITAL.CL,LAUTARO YANKAS 15265 CHILE Region Metropolita...,SAN BERNARDO,9941142745,79300098


In [55]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_7888\1036101616.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,1° Apellido Asegurado,2° Apellido Asegurado,apellidos,nacimiento,Edad,email,direccion,comuna,celular,rut
0,Comercial Maicao SPA,6985407.0,9,NaN,NaN,6985407-9,Mujer,Titular,NaN,RUTH,FLORES,FUENZALIDA,FLORES FUENZALIDA,1957-01-13,66.0,6985407@casilladigital.cl,STUGUTTGARD 2112 Chile Decimo Quinta Region A...,ARICA,993894545,69854079
1,Comercial Maicao SPA,7141749.0,2,NaN,NaN,7141749-2,Mujer,Titular,NaN,INELIA,RODRIGUEZ,LEDESMA,RODRIGUEZ LEDESMA,1959-07-23,63.0,rodriguezinelia@gmail.com,AVDA. VALPARAISO CASA 1170 3 CHILE Región de ...,QUILLOTA,95966963,71417492
2,Comercial Maicao SPA,7196473.0,6,NaN,NaN,7196473-6,Mujer,Titular,NaN,JULIA,BURGOS,BURGOS,BURGOS BURGOS,1956-05-20,66.0,7196473@casilladigital.cl,GRAJALES 2461 30 CHILE Region Metropolitana d...,SANTIAGO,NaN,71964736
3,Comercial Maicao SPA,7476959.0,4,NaN,NaN,7476959-4,Mujer,Titular,NaN,MARISOL,BRAVO,LUARTE,BRAVO LUARTE,1955-12-02,67.0,7476959@casilladigital.cl,ECUADOR 1560 Chile Cuarta Region ELQUI LA SERENA,LA SERENA,998378146,74769594
4,Comercial Maicao SPA,7930009.0,8,NaN,NaN,7930009-8,Mujer,Titular,NaN,FRESIA EUGENIA,TAHORMINA,OLIVER,TAHORMINA OLIVER,1958-12-19,64.0,7930009@CASILLADIGITAL.CL,LAUTARO YANKAS 15265 CHILE Region Metropolita...,SAN BERNARDO,9941142745,79300098


In [56]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_7888\2096211457.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_7888\2096211457.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [57]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(3186, 30)

In [58]:
Pacientes_con_id.head()

,Empresa,Rut Titular,Dv Titular,Rut Asegurado,Dv,rut1,sexo,Parentesco,Descripcion,nombres,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,Comercial Maicao SPA,6985407.0,9,NaN,NaN,6985407-9,Mujer,Titular,NaN,Ruth,...,True,0018c00002egpM4AAI,69854079,Ruth Flores Fuenzalida,RUT,None,None,None,Mujer,None
1,Comercial Maicao SPA,7141749.0,2,NaN,NaN,7141749-2,Mujer,Titular,NaN,Inelia,...,True,0018c00002egpM6AAI,71417492,Inelia Rodriguez Ledesma,RUT,None,None,None,Mujer,None
2,Comercial Maicao SPA,7196473.0,6,NaN,NaN,7196473-6,Mujer,Titular,NaN,Julia,...,True,0018c00002egpM7AAI,71964736,Julia Burgos Burgos,RUT,None,None,None,Mujer,None
3,Comercial Maicao SPA,7476959.0,4,NaN,NaN,7476959-4,Mujer,Titular,NaN,Marisol,...,True,0018c00002egpM8AAI,74769594,Marisol Bravo Luarte,RUT,None,None,None,Mujer,None
4,Comercial Maicao SPA,7930009.0,8,NaN,NaN,7930009-8,Mujer,Titular,NaN,Fresia Eugenia,...,True,0018c00002egpN5AAI,79300098,Fresia Eugenia Tahormina Oliver,RUT,None,None,None,Otros,None


In [59]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [60]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"]
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]
if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"]

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "Mexico"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000004cvS7EAI'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingStreet,BillingState,Phone,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
10,False,Hugo,Palma Cruz,,,1958-09-30,Hombre,NaN,NaN,NaN,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
11,False,Luis,Taiba Poblete,,,1973-06-04,Hombre,NaN,NaN,NaN,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
13,False,Ramon Roberto,Tejada,,,1962-08-31,Hombre,NaN,NaN,NaN,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
19,False,Marta Ines,Saavedra Arnao,87213064,titi1123.msa@gmail.com,1963-01-12,Mujer,Pasaje Cala 1899 Chile Region Metropolitana D...,Padre Hurtado,981571105,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE
33,False,Rodrigo,Barrera Pinones,,,1962-12-17,Hombre,NaN,NaN,NaN,0128c000002Byp4AAC,Chile,RUT,a1U8c000005dAtnEAE


In [61]:
    
clinica_virtual = pd.DataFrame()    
clinica_virtual['rut'] = Pacientes_con_id["rut1"].str.upper()
clinica_virtual['firstName'] = Pacientes_con_id['nombres']
clinica_virtual['lastName'] = Pacientes_con_id['apellidos']
clinica_virtual['accountId'] = campaña



Codigos_Clinica_Virtual = {
'BCI':'1',
'Farmacias Ahumada':'2',
'MetLife':'3',
'Vida Security':'4',
'Willis':'5',
'Entel':'6',
'Testing':'7',
'3SP':'8',
'HomeCare':'9',
'Pedro de Valdivia':'10',
'Colaboradores':'11',
'MineraAbra':'12',
'CMPC':'13',
'Lipigas':'14',
'Asismed':'15',
'Siempre Perú':'16',
'Farmacias Cruz Verde':'17',
'WFS Food Services':'18',
'VidaCel':'19',
'Dermik':'20',
'EY':'21',
'IND':'22',
'Sindicato Walmart':'23',
'Instruvalve':'24',
'VidaCel Colaboradores':'25',
'Socofar':'26',
'Milan Fajanovic':'27',
'THBseguros':'28',
'Pesco':'29',
'Burgos - Consorcio':'30',
'COFAMA-SILENTIUM':'31',
'KAUFMANN':'32',
'QuePlan':'33',
'Zurich':'34',
'Colaboradores Siempre Chile':'35',
'Talo':'36',
'TRANSVIP':'37',
'Vida Security Colaboradores':'38',
'FORUS':'39',
'SURA':'40',
'Southbrige':'41',
'Construmart':'42',
'BICE Vida':'43'

}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [62]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
160244,a1Y8c00000C6VgWEAV,0018c00002amR8oAAE,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,EMP7405,A-0729
160245,a1Y8c00000C6VgXEAV,0018c00002amQoMAAU,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,EMP7405,A-0729
160246,a1Y8c00000C6VgYEAV,0018c00002amQoNAAU,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,EMP7405,A-0729
160247,a1Y8c00000C6VgZEAV,0018c00002amQoQAAU,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,EMP7405,A-0729
160248,a1Y8c00000C6VgbEAF,0018c00002amQozAAE,a1W8c0000083iWpEAI,Socofar,0018c00002cXJTzAAO,Comercial Maicao SPA,Sin Holding Asociado,Socofar,Comercial Maicao SPA,Sin Póliza Asociada,EMP7405,A-0729


In [63]:
campaña_seleccionada.shape

(139, 12)

In [64]:

if campaña != 'MetLife':
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente"
    )
    pacientes_en_campaña['Paciente__c'] = 'Id Pacientes'
    pacientes_en_campaña['Programas_por_Empresas__c'] = 'Id Programas Por empresa'
    pacientes_en_campaña.to_excel("F:/Empresas SF/reportes/campaña_empresa.xlsx",index=False)

In [65]:
campaña_en_pacientes = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account"
)
campaña_en_pacientes.to_excel("F:/Empresas SF/reportes/archivo_campaña.xlsx",index=False)

In [66]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
